In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MSE
from tqdm import tqdm

## Import Dataset

In [ ]:
dataset_name = "cifar10"
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes

IMG_SIZE = 176
batch_size = 1
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

## Load Model

In [ ]:
model = load_model('./TinyML Models/mcunet/mcunet_320_1.h5')
model.summary()

## Attack

In [ ]:
def fgsm_attack(model, image, label, eps):
    image = tf.cast(image, tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(image)
        pred = model(image)
        loss = MSE(label, pred)
        gradient = tape.gradient(loss, image)
        signedGrad = tf.sign(gradient)
        adversary = (image + (signedGrad * eps)).numpy()
        return adversary

In [ ]:
eps = [0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.5,2.0,5.0,10.0,15.0,20.0,30.0,40.0,50.0]
accu_num = []
for item in eps:
    countadv = 0
    for image, label in tqdm(ds_test):
        imageLabel = np.array(label).argmax()
        imagePred = model.predict(image)
        imagePred = imagePred.argmax()
        adversary = fgsm_attack(model,image, label, eps=item)
        pred = model.predict(adversary)
        adversaryPred = pred[0].argmax()
        if imagePred == adversaryPred:
            countadv += 1

    print("Adversarial accuracy : ", countadv / len(ds_test))
    accu_num.append(countadv / len(ds_test))

In [ ]:
accu_num